In [13]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import IPython.display as ipd
from pathlib import Path

Establece la ruta del dataset

In [2]:
ruta="/Users/andavaro/Desktop/Andrès/UBA/TrabajoDeGradoCEIA/VENV_Python/Tesis_IA/Dataset/"

In [3]:
data_dir = pathlib.Path(ruta)
os.chdir(ruta)
default_dir = os.getcwd()
print(f'Data directory will be: {ruta}')

Data directory will be: /Users/andavaro/Desktop/Andrès/UBA/TrabajoDeGradoCEIA/VENV_Python/Tesis_IA/Dataset/


Importa los nombres de las carpetas que contiene el dataset

In [4]:
labels = [name for name in os.listdir('.') if os.path.isdir(name)]
# back to default directory
os.chdir(default_dir)
print(f'Total Labels: {len(labels)}')
print(f'Label Names: {labels}')

Total Labels: 3
Label Names: ['VT', 'Copia de VT 2', 'Copia de VT']


Calcula el total de muestras que contiene el dataset

In [5]:
filenames = tf.io.gfile.glob(str(default_dir) + '/*/*')
num_samples = len(filenames)
print('Number of total examples:', num_samples)

Number of total examples: 30


In [30]:
for l in labels:
    print(l)

VT
Copia de VT 2
Copia de VT


In [36]:
def load_audio_files(main_path: str, label:str):

    dataset = []
    
    for l in labels:
        
        dataset_l = []
        path = main_path+l
        walker = sorted(str(p) for p in Path(path).glob(f'*.csv'))

        for i, file_path in enumerate(walker):
            path, filename = os.path.split(file_path)
            
            file = pd.read_csv(path+'/'+filename)
            mean=np.mean(file.Señal)
            data = tf.convert_to_tensor(file.Señal-mean, dtype=tf.float32)
            
            dataset_l.append([data, l])

        dataset.append(dataset_l)
        
    return dataset

In [47]:
trainset = load_audio_files(ruta, labels)
print(f'Número de datasets: {len(trainset)}')
for ds in range(len(trainset)):
    print('Tamaño del dataset', trainset[ds][0][1], ': ', len(trainset[ds]))

Número de datasets: 3
Tamaño del dataset VT :  5
Tamaño del dataset Copia de VT 2 :  5
Tamaño del dataset Copia de VT :  5


In [59]:
def get_spectrogram(waveform, ns=300, sol=0.5):
    '''
    waveform = señal
    ns = número de muestras por sección
    sol = porcentaje de solapamiento
    '''
    frame_length = round(waveform.shape[0] / (ns-(ns-1)*sol))
    frame_step = round(sol*frame_length)
    
    #frame_length = 255
    #frame_step = 128
    # Padding for files with less than 16000 samples
    zero_padding = tf.zeros([waveform.shape[0]] - tf.shape(waveform), dtype=tf.float32)

    # Concatenate audio with padding so that all audio clips will be of the same length
    waveform = tf.cast(waveform, tf.float32)
    equal_length_waveform = tf.concat([waveform, zero_padding], 0)
    
    # Option 1: Use tfio to get the spectrogram
    #spect = tfio.audio.spectrogram(input=equal_length_waveform, nfft=frame_length, window=frame_length, stride=frame_step)
    
    # Option 2: Use tf.signal processing to get the Short-time Fourier transform (stft)
    spectro = tf.signal.stft(equal_length_waveform, frame_length=frame_length, frame_step=frame_step, window_fn=tf.signal.hamming_window)
    spectrogram = tf.abs(spectro)

    return spectrogram#, spect


In [57]:
def create_images(dataset, ds_dir, label_dir):
    # make directory
    for l in range(len(label_dir)):
        directory = f'{ds_dir}{label_dir[l]}/'

        os.makedirs(directory, mode=0o777, exist_ok=True)

        for i, data in enumerate(dataset[l]):

            waveform = data[0]
            spectrogram= get_spectrogram(waveform)

            # Split test and train images by 30%
            
            plt.imsave(f'{ds_dir}{label_dir[l]}/spec_img{i}.png', spectrogram.numpy().T)
            

        

In [60]:
ns = 300
sol = 0.7
lf = 0
uf = 250
create_images(trainset, ruta, labels)